In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np

# 1. Загрузка данных
data = pd.read_csv('processed_data.csv')  # путь к вашему файлу с данными

# 2. Предобработка данных
categories = [
    'Вопрос решен',
    'Нравится качество выполнения заявки',
    'Нравится качество работы сотрудников',
    'Нравится скорость отработки заявок',
    'Понравилось выполнение заявки',
    'Другое'
]

labels = data[categories].values.astype(int)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['comment'].values,
    labels,
    test_size=0.2,
    random_state=42
)

# 3. Токенизация и подготовка датасета
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class CommentsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.FloatTensor(label)
        }

train_dataset = CommentsDataset(train_texts, train_labels, tokenizer)
val_dataset = CommentsDataset(val_texts, val_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# 4. Модель с несколькими выходами (multi-label)
class BertMultiLabelClassifier(nn.Module):
    def __init__(self, dropout=0.3):
        super(BertMultiLabelClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.bert.config.hidden_size, len(categories))
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits= self.classifier(pooled_output)
        return logits

model= BertMultiLabelClassifier()
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 5. Обучение и валидация с расчетом F1-метрики
criterion= nn.BCEWithLogitsLoss()
optimizer= optim.Adam(model.parameters(), lr=2e-5)

num_epochs=3

for epoch in range(num_epochs):
    model.train()
    total_loss=0
    for batch in train_loader:
        input_ids= batch['input_ids'].to(device)
        attention_mask= batch['attention_mask'].to(device)
        labels= batch['labels'].to(device)

        optimizer.zero_grad()
        outputs= model(input_ids=input_ids, attention_mask=attention_mask)
        loss= criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss+= loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}")

# Валидация и расчет F1 для каждого класса
model.eval()
all_preds=[]
all_true=[]

with torch.no_grad():
    for batch in val_loader:
        input_ids= batch['input_ids'].to(device)
        attention_mask= batch['attention_mask'].to(device)
        labels= batch['labels'].cpu().numpy()

        outputs= model(input_ids=input_ids, attention_mask=attention_mask)
        preds= torch.sigmoid(outputs).cpu().numpy()

        all_preds.extend(preds)
        all_true.extend(labels)

# Преобразуем вероятности в бинарные метки по порогу 0.5
pred_labels=np.array(all_preds) >= 0.5
true_labels=np.array(all_true)

# Расчет F1 для каждого класса и среднего значения
for i, category in enumerate(categories):
    f1_macro=f1_score(true_labels[:,i], pred_labels[:,i], average='macro')
    print(f"F1-score для '{category}': {f1_macro:.4f}")

# Также можно вывести средний F1 по всем категориям:
f1_avg=f1_score(true_labels,pred_labels,average='macro')
print(f"\nСредний F1-score по всем категориям: {f1_avg:.4f}")

c:\Users\Игорь.А\OneDrive\Документы\GitHub\Intensive_4\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Игорь.А\OneDrive\Документы\GitHub\Intensive_4\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Игорь.А\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python

Epoch 1/3, Loss: 0.5039
Epoch 2/3, Loss: 0.4392
Epoch 3/3, Loss: 0.3617
F1-score для 'Вопрос решен': 0.5477
F1-score для 'Нравится качество выполнения заявки': 0.4826
F1-score для 'Нравится качество работы сотрудников': 0.7484
F1-score для 'Нравится скорость отработки заявок': 0.9121
F1-score для 'Понравилось выполнение заявки': 0.4640
F1-score для 'Другое': 0.8037

Средний F1-score по всем категориям: 0.4038
